# Motzkin

Tabla de 1 dimensión y tamaño $n+1$ ya que tenemos que resolver $M(0),\dots,M(n)$ subproblemas.

Sustituyendo para $M(4),\dots,M(0)$ en la ecuación de recurrencia:
- $M(4)$ depende de todos los anteriores.
- $M(3)$ depende de todos los anteriores.
- $M(2)$ depende de todos los anteriores.
- $M(0)$ y $M(1)$ son los casos base.

In [1]:
def motzkin(n, memo):
    if n in memo:
        return memo[n]
    if n == 0 or n == 1:
        r = 1
    else:
        r = motzkin(n - 1, memo)
        for i in range(0, n - 1):
            r += motzkin(i, memo)*motzkin(n - 2 - i, memo)
    memo[n] = r
    return r

In [2]:
memo = {}
motzkin(10, memo)

2188

El orden de este algoritmo es $\Theta(n^2)$. Ya que resolvemos $M(0),\dots,M(n)$, donde
- $M(0)$ y $M(1)$ son casos base y por tanto el tiempo es constante.
- $M(i)$ con $i\geq2$, tenemos un bucle de tamaño $i-1$. Así pues $\Theta(i)$.

$$\Theta(1) + \Theta(1) + \sum_{i=2}^n\Theta(i) = \Theta(n^2)$$

# ¿Existe un camino en un tablero?

$C(i,j)=$
- ($\texttt{down}[i-1][j]=\texttt{True}$ and $C(i-1,j)$) ó ($\texttt{right}[i][j-1]=\texttt{True}$ and $C(i,j-1)$); si $i>0$ y $j >0$.
- $\texttt{right}[i][j-1]=\texttt{True}$ and $C(i,j-1)$ si $i=0$
- $\texttt{down}[i-1][j]=\texttt{True}$ and $C(i-1,j)$ si $j=0$
- $\texttt{True}$ cuando $(i,j)=(0,0)$.

In [1]:
def existe_camino(down, right, i, j, memo):
    if (i,j) in memo:
        return memo[(i,j)]

    if i == 0 and j==0:
        r = True
    else:
        r = False
        if j > 0:
            r = r or right[i][j-1] and existe_camino(down, right, i, j-1, memo)
        if i > 0:
            r = r or down[i-1][j] and existe_camino(down, right, i-1, j, memo)
    memo[(i,j)] = r
    return r

In [2]:
R_existe = [
    [True, True, True],  
    [False, False, True],  
    [True, True, False], 
    [False, False, True],  
]

D_existe = [
    [True,  True, True, False],  
    [True, False,  False, True],
    [True, True,  True, False]
]


In [3]:
memo = {}
existe_camino(D_existe, R_existe, 3, 3, memo)

True

In [4]:
R_no_existe = [
    [True, True, True],  
    [False, False, True], 
    [False, True, False], 
    [True, False, True],  
]

D_no_existe = [
    [True,  True, True, False], 
    [True, False,  False, True],
    [True, True,  True, False]
]

In [5]:
memo = {}
existe_camino(D_no_existe, R_no_existe, 3, 3, memo)

False

In [6]:
def recuperar(memo, n, m, right):
    i = n
    j = m

    if not memo[(n,m)]:
        return "No hay sol"
    S = [(i,j)]
    while (i,j) != (0,0):
        if j > 0:
            if right[i][j-1] and memo[i, j-1]:
                j = j - 1
            else:
                i = i - 1
        else:
            i = i - 1
        S.append((i,j))
    return S[::-1]

In [8]:
memo = {}
existe_camino(D_existe, R_existe, 3, 3, memo)
recuperar(memo, 3, 3, R_existe)

[(0, 0), (1, 0), (2, 0), (2, 1), (2, 2), (3, 2), (3, 3)]

In [9]:
memo = {}
existe_camino(D_no_existe, R_no_existe, 3, 3, memo)
recuperar(memo, 3, 3, R_no_existe)

'No hay sol'

# Delannoy

Tabla de 2 dimensiones y tamaño $(n+1)\times(m+1)$ ya que tenemos que resolver $D(i,j)$ para $i=0,\dots,n$ y $j=0,\dots,m$ subproblemas.

Mismo DAG que en Manhattan pero añadiendo aristas en diagonal. Los casos base están en la primera fila y en la primera columna.

In [10]:
def delannoy(n, m, memo):
    if (n,m) in memo:
        return memo[(n,m)]
    if n == 0 or m == 0:
        r = 1
    else:
        r = delannoy(n-1, m, memo) + delannoy(n, m-1, memo) + delannoy(n-1, m-1, memo)
    memo[(n,m)] = r
    return r

In [12]:
memo = {}
delannoy(6, 6, memo)

8989

Para el orden, mismo razonamiento que en Manhattan. Cada subproblema se resuelve en $\Theta(1)$ y tenemos $\Theta(nm)$ subproblemas. Por tanto el orden es $\Theta(nm)$.

# Rana 🐸

$D(i)=$
- $\min(|h(i) - h(i-1)| + D(i-1), |h(i) - h(i-2)| + D(i-2))$ si $i\geq2$
- 0 si $i=0$
- $|h(1)-h(0)|$ si $i = 1$

In [20]:
def rana(h, n, memo):
    if n in memo:
        return memo[n]
    if n == 0:
        r = 0
    elif n == 1:
        r = abs(h[1] - h[0])
    else:
        r = min(abs(h[n] - h[n-1]) + rana(h, n-1, memo), 
                abs(h[n] - h[n-2]) + rana(h, n-2, memo))
    memo[n] = r
    return r

In [21]:
h = [10, 30, 40, 20, 50, 60]
memo = {}
rana(h, len(h)-1, memo)

50

In [22]:
def reconstruccion(memo, h, n):
    x_actual = n
    S = [n]
    while x_actual != 0:
        valor = memo[x_actual]
        if valor == abs(h[x_actual] - h[x_actual-1]) + memo[x_actual-1]:
            S.append(x_actual - 1)
            x_actual -= 1
        else:
            S.append(x_actual - 2)
            x_actual -= 2
    return S[::-1]

In [23]:
reconstruccion(memo, h, len(h) - 1)

[0, 1, 2, 4, 5]